In [ ]:
import torch
import torch.nn as nn
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from transformers import BeitForImageClassification
from sklearn.metrics import classification_report, confusion_matrix
from google.colab import drive

drive.mount('/content/drive')

IMAGE_SIZE = 224
BATCH_SIZE = 8
NUM_WORKERS = 2
NUM_CLASSES = 9
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", DEVICE)

val_transforms = transforms.Compose([
    transforms.Resize(IMAGE_SIZE),
    transforms.CenterCrop(IMAGE_SIZE),
    transforms.ToTensor(),
    transforms.Normalize([0.5]*3, [0.5]*3)
])

val_dataset = datasets.ImageFolder('/content/drive/MyDrive/dataset-dapa/test/', transform=val_transforms)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS, pin_memory=True)

model = BeitForImageClassification.from_pretrained(
    "microsoft/beit-base-patch16-224",
    num_labels=NUM_CLASSES,
    ignore_mismatched_sizes=True
)

model.classifier = nn.Sequential(
    nn.Dropout(0.3),
    nn.Linear(model.classifier.in_features, NUM_CLASSES)
)

model.load_state_dict(torch.load('/content/drive/MyDrive/BEiT-15epoch.pth', map_location=DEVICE))
model.to(DEVICE)
model.eval()

def evaluate_model(model, loader, device, class_names):
    correct = 0
    total = 0
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, preds = torch.max(outputs.logits, 1)

            total += labels.size(0)
            correct += (preds == labels).sum().item()

            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    acc = 100 * correct / total
    print(f"\nOverall Accuracy: {acc:.2f}%\n")

    report = classification_report(all_labels, all_preds, target_names=class_names, output_dict=True)
    cm = confusion_matrix(all_labels, all_preds)
    per_class_acc = cm.diagonal() / cm.sum(axis=1)

    print(f"{'Class':<25} {'Precision':<10} {'Recall':<10} {'F1-Score':<10} {'Class Acc':<10}")
    print("-" * 70)
    for idx, class_name in enumerate(class_names):
        cls = report[class_name]
        print(f"{class_name:<25} {cls['precision']:<10.2f} {cls['recall']:<10.2f} {cls['f1-score']:<10.2f} {per_class_acc[idx]*100:<10.2f}")

evaluate_model(model, val_loader, DEVICE, val_dataset.classes)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Device: cuda


Some weights of BeitForImageClassification were not initialized from the model checkpoint at microsoft/beit-base-patch16-224 and are newly initialized because the shapes did not match:
- classifier.bias: found shape torch.Size([1000]) in the checkpoint and torch.Size([9]) in the model instantiated
- classifier.weight: found shape torch.Size([1000, 768]) in the checkpoint and torch.Size([9, 768]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



Overall Accuracy: 95.28%

Class                     Precision  Recall     F1-Score   Class Acc 
----------------------------------------------------------------------
algal_spot                0.99       0.97       0.98       97.06     
brown_blight              0.85       0.99       0.91       98.51     
gray_blight               0.98       0.87       0.92       86.50     
healthy                   0.91       1.00       0.95       100.00    
helopeltis                1.00       0.93       0.96       92.67     
red-rust                  0.88       0.92       0.90       91.67     
red-spider-infested       1.00       1.00       1.00       100.00    
red_spot                  1.00       0.97       0.99       97.09     
white-spot                1.00       1.00       1.00       100.00    
